In [1]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.edge.options import Options as EdgeOptions
from selenium.webdriver.common.by import By
import pandas as pd
import numpy as np
import time

In [2]:
#출력 파일 이름, csv행 <- 제어 편의를 위해 일부로 xpath키랑 맞춤
result_file_name = 'klas_data.csv'
csv_column = ['강의제목', '과목개요', '학습목표']

In [3]:
#판별자
#해당 경로의 텍스트가 '외국어 강의' 면 외국어강의 항목이 있다
diff_element = '//*[@id="appModule"]/div[2]/div[2]/table[2]/tbody/tr[2]/th'

#가져올 요소의 주소 <- 외부키는 조건문편의를 위해 웹 출력값 그대로 이용
#                     내부키는 제어 편의를 위해 csv행과 맞춤
target_xpath = {
    '외국어 강의' : {
        csv_column[0] : '//*[@id="appModule"]/div[2]/div[2]/table[1]/tbody/tr[1]/td[1]' ,
        csv_column[1] : '//*[@id="appModule"]/div[2]/div[2]/table[2]/tbody/tr[4]/td',
        csv_column[2] : '//*[@id="appModule"]/div[2]/div[2]/table[2]/tbody/tr[6]/td'
    },
    '교과목\n개요' : {
        csv_column[0] : '//*[@id="appModule"]/div[2]/div[2]/table[1]/tbody/tr[1]/td[1]',
        csv_column[1] : '//*[@id="appModule"]/div[2]/div[2]/table[2]/tbody/tr[2]/td',
        csv_column[2] : '//*[@id="appModule"]/div[2]/div[2]/table[2]/tbody/tr[4]/td'
    }
}

In [4]:
#연도, 학기
year, term = "2025", '1'
#가져올 강의의 학정 번호
hakjeong_list  = [
    "6030-2-1821-01",
    "6030-3-1862-01",
    "6030-3-1862-02",
    "6030-1-3806-01",
    "6030-1-6721-01",
    "6030-2-0696-01",
    "6030-2-0779-01",
    "6030-2-0820-01",
    "6030-2-1186-01",
    "6030-2-1234-01",
    "6030-2-1506-01",
    "6030-2-9639-01",
    "6030-3-0878-01",
    "6030-3-1120-01",
    "6030-3-4264-01",
    "6030-3-5694-01",
    "6030-3-8504-01",
    "6030-3-9640-01",
    "6030-4-0410-01",
    "6030-4-0854-01",
    "6030-4-3190-01",
    "6030-4-4883-01",
    "6030-4-6544-01"
]


In [5]:
#연도, 학기, 학정번호 이용 강의계획서 url반환
def make_url(year, term, hakjeong):
    major, grade, num, section = hakjeong.split("-")
    base = 'https://klas.kw.ac.kr/std/cps/atnlc/popup/LectrePlanStdView.do?selectSubj=U'
    return base + year + term + num + major + section + grade

In [6]:
# 함수: XPath에서 텍스트 추출 (공란이면 None)
def get_text_or_nan(driver, xpath):
    try:
        txt = driver.find_element(By.XPATH, xpath).text.strip()
        return txt if txt else None
    except:
        return None

In [7]:
driver = webdriver.Chrome()

# 수동 로그인 (한 번 로그인하면 쿠키 유지됨)
driver.get("https://klas.kw.ac.kr")
input("KLAS에 로그인한 후 엔터를 누르세요...")

# 새 빈 탭으로 이동
driver.execute_script("window.open();")
time.sleep(1)
driver.switch_to.window(driver.window_handles[-1])
time.sleep(1.5)

# 크롤링 결과를 저장, 데이터프레임으로 만들 딕셔너리
data = {i:[] for i in csv_column}

# 학정번호마다 크롤링 실행
for hakjeong in hakjeong_list:

    url = make_url(year, term, hakjeong)

    driver.get(url)
    time.sleep(1)
    
    for k, v in target_xpath[get_text_or_nan(driver, diff_element)].items():
        data[k].append(get_text_or_nan(driver, v))

# 데이터프레임으로 변환하고 CSV로 저장
df = pd.DataFrame(data)
df.to_csv(result_file_name, index=False, encoding = 'cp949')

print("✅ klas_data.csv 저장 완료!")

✅ klas_data.csv 저장 완료!
